##This file is to check out reactions of the model P. acidipropionicum 
model name: PAC_4875_model.xml

workflow: 

1.   Go to MEMOTE and find list of unbalanced reactions (eg: https://memote.io/report/92feb776-c3cf-4fd1-a06a-9f8cbbbaeb45)
2.   split list into reactions missing formulas and unbalanced reactions (i.e., with all formulas present in the reaction)
3. loop over "unbalanced reactions" make program to find unbalanced element in reaction and fix 
4. go over reactions with missing formulas and try find them (idk if this part is essential)

wat I still need to ask: 1) wtf is PRS or X after a formula, 2) can I leave unbalanced reactions due to miussing formulas alone... 3) is it acceptable to ignore is lacking reactions if 97% of all the reactions are ok? 




In [ ]:
!setup.py install
!pip install cobra

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# import modules 
import cobra.io
import os
from cobra.flux_analysis import flux_variability_analysis
import matplotlib.pyplot as plt
import numpy as np

usefull functions 

In [5]:
def showFormula(rnx):
  reactants = rnx.reactants
  products = rnx.products

  print('reactants')
  for i in reactants: 
    if i.formula: 
      print(i.formula)
    else: 
      print('missing')

  print('')
  print('products')
  for i in products:
    if i.formula: 
      print(i.formula)
    else: 
      print('missing') 

def writeReactionInFormulas(rxn):
  reactants = rxn.reactants
  products = rxn.products
  eq = ''
  for i in reactants: 
    if i.formula: 
       eq += ' + {}'.format(i.formula)
  eq += ' = '
  for i in products:
    if i.formula: 
      eq += ' + {}'.format(i.formula)
  return eq 

#Analyze model

In [10]:
model = cobra.io.read_sbml_model('/content/drive/MyDrive/Colab Notebooks/models/PAC_4875_model.xml') 
unbalancedRnx = ["rxnnew81_c0","rxnnew85_c0","rxnnew52_c0","rxnnew54_c0","rxnnew55_c0","rxnnew56_c0","rxnnew57_c0","rxnnew58_c0","rxnnew59_c0","rxnnew60_c0","rxnnew61_c0","rxnnew62_c0","rxnnew103_c0","rxnnew63_c0","rxnnew104_c0","rxnnew64_c0","rxnnew65_c0","rxnnew66_c0","rxnnew67_c0","rxnnew69_c0","rxnnew70_c0","rxnnew71_c0","rxnnew72_c0","rxnnew73_c0","rxnnew74_c0","rxnnew75_c0","rxn06118_c0","rxn06120_c0","rxnnew04_c0","rxnnew06_c0","rxnnew09_c0","rxn05890_c0","rxnnew12_c0","rxn06651_c0","rxnnew48_c0"]
print('there are {} reactions which are unbalanced or have missing formulas'.format(len(unbalancedRnx)))       
print('')

#have a look at the model summary 
opt = model.optimize()
model.summary()

there are 35 reactions which are unbalanced or have missing formulas



Metabolite,Reaction,Flux,C-Number,C-Flux
S_cpd00009_ext,Ex_S_cpd00009_ext,1.138,0,0.00%
S_cpd00013_ext,Ex_S_cpd00013_ext,7.973,0,0.00%
S_cpd00027_ext,Ex_S_cpd00027_ext,10,6,99.92%
S_cpd00030_ext,Ex_S_cpd00030_ext,0.0005644,0,0.00%
S_cpd00048_ext,Ex_S_cpd00048_ext,0.003532,0,0.00%
S_cpd00099_ext,Ex_S_cpd00099_ext,0.004241,0,0.00%
S_cpd00104_ext,Ex_S_cpd00104_ext,0.003806,10,0.06%
S_cpd00149_ext,Ex_S_cpd00149_ext,0.0006082,0,0.00%
S_cpd00205_ext,Ex_S_cpd00205_ext,0.159,0,0.00%
S_cpd00239_ext,Ex_S_cpd00239_ext,0.1706,0,0.00%


In [36]:
#check out c-flux in g-C /g-DW/h
mmC = 12
gCglu = 10*6*mmC
gCprop = 6.487*3*mmC 
print(gCprop/gCglu)

0.32434999999999997


In [20]:
# analyse the outputs do the biomass reaction and mass balances make sense
model.metabolites.get_by_id('S_biomass_ext').summary()
#df = model.summary()

Percent,Flux,Reaction,Definition
100.00%,0.8562,biomass_c0,0.038 S_cpd11461_c0 + 0.434 S_cpd11463_c0 + 0.176 S_cpd11613_c0 + 0.03 S_cpdnew18_c0 + 0.032 S_cpdnew26_c0 + 0.2 S_cpdnew27_c0 --> S_biomass_ext
Percent,Flux,Reaction,Definition
100.00%,-0.8562,Ex_S_biomass_ext,S_biomass_ext -->


In [28]:
model.metabolites.get_by_id('S_cpd00011_ext').summary()

Percent,Flux,Reaction,Definition
100.00%,1.497,rxn05467_c0,S_cpd00011_ext <=> S_cpd00011_c0
Percent,Flux,Reaction,Definition
100.00%,-1.497,Ex_S_cpd00011_ext,S_cpd00011_ext -->


In [ ]:
# look at an e.g of an unblanced reaction
reactionSelection = unbalancedRnx[25]
# prints the reation dataframe summary 
model.reactions.get_by_id(reactionSelection)

In [ ]:
# get formulas of above example
rnx = model.reactions.get_by_id(reactionSelection)
showFormula(rnx)

In [18]:
#make a list of all unbalanced metabolic reactions
metabolic_rct = []
for rct in model.reactions:
    if rct.id in unbalancedRnx:
        if rct.id[-1:] in 't':
            continue
        else: 
            metabolic_rct.append(rct)
    else: continue
len(metabolic_rct)    #needs to be 35

35

In [40]:
# check how many reactions have missing formulas 
RnxWithMissingFormulas = []
#wrongStoichiometryRnx = []
for rnx in metabolic_rct:
    for met in rnx.metabolites.keys():
      if not met.formula: 
        RnxWithMissingFormulas.append(rnx)
        break
      else:continue 
        #wrongStoichiometryRnx.append(rnx)
        #break

wrongStoichiometryRnx = list(  set(metabolic_rct) - set(RnxWithMissingFormulas) )
print(len(RnxWithMissingFormulas))
print(len(wrongStoichiometryRnx) )
print(len(RnxWithMissingFormulas) + len(wrongStoichiometryRnx)  )

18
17
35


## We see 18 of the 35 reactions that are un balanced are due to missing formulas, how can we we resolve this? 

In [63]:
rxn = wrongStoichiometryRnx[0]
showFormula(rxn)
eq = writeReactionInFormulas(wrongStoichiometryRnx[0]) # wtf why is this compleetly wrong
print('')
print(eq)
print('')
# print the summary of the reaction 
wrongStoichiometryRnx[0]

reactants
O2
C5H7O4

products
CO2
H
C4H6O2

 + O2 + C5H7O4 =  + CO2 + H + C4H6O2



Reaction identifier,rxnnew85_c0
Name,Spontaneous diacetyl formation
Memory address,0x7fb3c47fddd0
Stoichiometry,S_cpd00007_c0 + S_cpd00668_c0 --> S_cpd00011_c0 + S_cpd00067_c0 + S_cpd00551_c0 O2 + ALCTT --> CO2 + H+ + DTYL
GPR,SPONTANEOUS
Lower bound,0.0
Upper bound,1000.0


In [61]:
model.metabolites.get_by_id('S_cpd00551_c0') # diacetyl (C4H6O2)
model.metabolites.get_by_id('S_cpd00668_c0')  # alfa-acetolactic acid (C5H7O4)

Metabolite identifier,S_cpd00668_c0
Name,ALCTT
Memory address,0x7fb3c55f0f90
Formula,C5H7O4
Compartment,c
In 5 reaction(s),"rxnnew85_c0, rxn02187_c0, rxn02186_c0, rxn02185_c0, rxn02113_c0"


i think I'll just leave this reaction alone 

next reaction

In [64]:
rxn = wrongStoichiometryRnx[1]
showFormula(rxn)
eq = writeReactionInFormulas(wrongStoichiometryRnx[1]) # wtf why is this compleetly wrong
print('')
print(eq)
print('')
# print the summary of the reaction 
wrongStoichiometryRnx[1]

reactants
C10H13N5O13P3
HO10P3

products
C10H13N5O10P2
H
HO13P4

 + C10H13N5O13P3 + HO10P3 =  + C10H13N5O10P2 + H + HO13P4



Reaction identifier,rxnnew04_c0
Name,ATP:polyphosphate phosphotransferase
Memory address,0x7fb3c4bb5fd0
Stoichiometry,S_cpd00002_c0 + S_cpd00421_c0 <=> S_cpd00008_c0 + S_cpd00067_c0 + S_cpdnew01_c0 ATP + Triphosphate <=> ADP + H+ + Polyphosphate
GPR,CDS.2695
Lower bound,-1000.0
Upper bound,1000.0


ATP:polyphosphate phosphotransferase  
The reaction stoichiometry is correct but the forumals for ATP and ADP seem wrong should be C10H16N5O13P3 (ATP) and C10H15N5O10P2 (ADP)  
=> will this change fuck up other reactions?? 

In [66]:
model.metabolites.get_by_id('S_cpd00002_c0') # ATP 
model.metabolites.get_by_id('S_cpd00008_c0')  # ADP

Metabolite identifier,S_cpd00008_c0
Name,ADP
Memory address,0x7fb3c589bb50
Formula,C10H13N5O10P2
Compartment,c
In 178 reaction(s),"rxn01513_c0, rxn05192_c0, rxn03406_c0, rxn00119_c0, rxn05512_c0, rxn00172_c0, rxn05539_c0, rxn05535_c0, rxn00744_c0, rxn03147_c0, rxn01509_c0, rxn00689_c0, rxn10571_c0, rxn01607_c0, rxn01653_c0,..."


Yes, changing the formula for ATP or ADP will change the stoichiometry of 178 other reactions!!??

In [70]:
strR = 'rxn01653_c0' # rxn01513_c0
rxn  = model.reactions.get_by_id(strR)
print(rxn in metabolic_rct)
showFormula(rxn)
eq = writeReactionInFormulas(wrongStoichiometryRnx[1]) # wtf why is this compleetly wrong
print('')
print(eq)
print('')
# print the summary of the reaction 
model.reactions.get_by_id(strR)

False
reactants
C10H13N5O13P3
C20H21N7O7

products
C10H13N5O10P2
HO4P
C20H20N7O6

 + C10H13N5O13P3 + HO10P3 =  + C10H13N5O10P2 + H + HO13P4



Reaction identifier,rxn01653_c0
Name,5-Formyltetrahydrofolate cyclo-ligase (ADP-forming) c0
Memory address,0x7fb3c4d32210
Stoichiometry,S_cpd00002_c0 + S_cpd02197_c0 --> S_cpd00008_c0 + S_cpd00009_c0 + S_cpd00347_c0 ATP + 5-Formyltetrahydrofolate --> ADP + Phosphate + 5-10-Methenyltetrahydrofolate
GPR,CDS.948
Lower bound,0.0
Upper bound,1000.0


# Check out which elements (C, N, O or P) are unbalanced 

In [25]:
from cobra import Reaction
#calc Number of each element in the metabolite in this reaction
C_unbalanced =[]
O_unbalanced =[]
N_unbalanced =[]
P_unbalanced =[]
H_unbalanced = [] # added 
error_formula = []

for rct in metabolic_rct:
#make one dictionary that contains the information about the total elements involved in each metabolite of the whole reaction
    rct_dict = {}
    all_H = []
    all_C = []
    all_O = []
    all_N = []
    all_P = []
    stoich = {}
    for met in rct.metabolites.keys():
        met_formula = met.elements
        if not met.elements:
            error_formula.append(met)
        else:
            try:
                H = met_formula['H']
            except KeyError:
                H = 0
            try:
                C = met_formula['C']
            except KeyError:
                C = 0
            try: 
                O = met_formula['O']
            except KeyError:
                O = 0
            try: 
                N = met_formula['N']
            except KeyError:
                N = 0
            try: 
                P = met_formula['P']
            except KeyError:
                P = 0
        elements = {f"H_{met.id}":H,f"C_{met.id}":C, f"O_{met.id}":O,f"N_{met.id}":N,f"P_{met.id}":P}
        rct_dict.update(elements)
#make a dictionary that has the information about the met.id and stoichiometry
        for met in rct.metabolites.keys():
            stoichiometry = rct.metabolites.get(met)
            stoich[met.id]=stoichiometry
        #make a calculation for the final balance of hydrogen for example:
    for key in rct_dict:
        id_key = stoich[key[2:]]
        el = key[:2]
        if el in 'H_':
            num_H = rct_dict.get(key)
            totH_key = num_H * id_key
            all_H.append(totH_key)
        elif el in 'C_':
            num_C = rct_dict.get(key)
            totC_key = num_C * id_key
            all_C.append(totC_key)
        elif el in 'O_':
            num_O = rct_dict.get(key)
            totO_key = num_O * id_key
            all_O.append(totO_key)
        elif el in 'N_':
            num_N = rct_dict.get(key) 
            totN_key = num_N * id_key
            all_N.append(totN_key)
        elif el in 'P_':
            num_P = rct_dict.get(key)
            totP_key = num_P * id_key
            all_P.append(totP_key)
        else: 
            continue
#add the number of that element to the list, and sum all parts of the list. this sum should equal 0 or it is unbalanced
    balance_H = sum(all_H) 
    balance_C = sum(all_C)
    balance_O = sum(all_O)
    balance_N = sum(all_N)
    balance_P = sum(all_P)
    if balance_H != 0 and balance_C == 0 and balance_O == 0:
      H_unbalanced.append(rct)
    # #Make a script to balance the reaction with a proton)
    #     add_H = -sum(all_H) #i.e. the stoichiometry of the proton needed to get mass balance
    #     proton = model.metabolites.h_c
    #     rct.add_metabolites({proton:add_H})
    if balance_C!= 0:
        C_unbalanced.append(rct)
    if balance_O != 0:
        O_unbalanced.append(rct)
    if balance_N != 0:
        N_unbalanced.append(rct)
    if balance_P != 0:
        P_unbalanced.append(rct)
   

In [26]:
print('Number of Hydrogen unbalanced rcts:', len(H_unbalanced))
print('Number of carbon unbalanced rcts:', len(C_unbalanced))
print('Number of oxygen unbalanced rcts:', len(O_unbalanced))
print('Number of nitrogen unbalanced rcts:', len(N_unbalanced))
print('Number of phosphorous unbalanced rcts:', len(P_unbalanced))
# print(rct_dict)
# print(H_unbalanced)

Number of Hydrogen unbalanced rcts: 11
Number of carbon unbalanced rcts: 22
Number of oxygen unbalanced rcts: 21
Number of nitrogen unbalanced rcts: 19
Number of phosphorous unbalanced rcts: 13


##Lets first have a look at the reactions where the formulas are missing 

In [27]:
print('there are {} metabolites that do not a have a formula in {} reactions'.format(len(error_formula), len(RnxWithMissingFormulas) ))

NameError: ignored

In [60]:
#lets have a look at an example 
error_formula[4]

Metabolite identifier,S_cpdnew10_c0
Name,average fatty acyl-ACP
Memory address,0x7fbd0059c410
Formula,
Compartment,c
In 6 reaction(s),"rxnnew58_c0, rxnnew55_c0, rxnnew54_c0, rxnnew60_c0, rxnnew59_c0, rxnnew57_c0"


In [39]:
#	let have a look at the reations involved with metabolite S_cpdnew10_c0 (average fatty acyl-ACP) which has no formula 
rnxMet = ['rxnnew58_c0', 'rxnnew55_c0', 'rxnnew54_c0', 'rxnnew60_c0', 'rxnnew59_c0', 'rxnnew57_c0']
model.reactions.get_by_id(rnxMet[0])

Reaction identifier,rxnnew58_c0
Name,Lysophosphatidylinositol synthesis
Memory address,0x7fbcff567d10
Stoichiometry,S_cpd00001_c0 + S_cpd00002_c0 + S_cpd00052_c0 + S_cpd00080_c0 + S_cpd00863_c0 + S_cpdnew10_c0 --> S_cpd00009_c0 + 2.0 S_cpd00012_c0 + S_cpd00018_c0 + S_cpd00046_c0 + S_cpd11493_c0 + S_cpdnew14_c0 H2O + ATP + CTP + Glycerol-3-phosphate + beta-D-Glucose 6-phosphate + average fatty acyl-ACP --> Phosphate + 2.0 PPi + AMP + CMP + ACP + Avg. Lysophosphatidylinositol
GPR,CDS.1625 and CDS.2164
Lower bound,0.0
Upper bound,1000.0


In [48]:
showFormula(model.reactions.get_by_id(rnxMet[0]))

reactants
H2O
C10H13N5O13P3
C9H13N3O14P3
C3H8O6P
C6H12O9P
missing

products
HO4P
H2O7P2
C10H13N5O7P
C9H13N3O8P
C11H21N2O7PRS
missing


##let have a look at reaction where all elements (C, N, P, & O) are unbalanced 

In [49]:
#overlay = list(set(H_unbalanced) & set(C_unbalanced) & set(O_unbalanced) & set(N_unbalanced) & set(P_unbalanced))
overlay = list(set(C_unbalanced) & set(O_unbalanced) & set(N_unbalanced) & set(P_unbalanced))
print(len(overlay))

12


ok so there are 12 reactions where it all went wrong (C unbalanced O unbalanced N unbalanced P unbalanced ), lets have a look at one 



In [54]:
nr = 9

In [55]:
overlay[nr] 

Reaction identifier,rxnnew61_c0
Name,Diacylisonisotl mannoside synthesis
Memory address,0x7fbcff567950
Stoichiometry,S_cpd00083_c0 + S_cpdnew14_c0 --> S_cpd00031_c0 + S_cpd00067_c0 + S_cpdnew17_c0 GDP-mannose + Avg. Lysophosphatidylinositol --> GDP + H+ + Avg. Diacylinositol mannoside
GPR,Open_problem
Lower bound,0.0
Upper bound,1000.0


In [56]:
showFormula(overlay[nr])

reactants
C16H23N5O16P2
missing

products
C10H13N5O11P2
H
missing


In [46]:
model.metabolites.get_by_id('S_cpdnew19_c0')

Metabolite identifier,S_cpdnew19_c0
Name,"UDP-2,3-diacetamido-2,3-dideoxy-D-mannuronate"
Memory address,0x7fbd005292d0
Formula,
Compartment,c
In 2 reaction(s),"rxnnew64_c0, rxnnew63_c0"


In [80]:
# look for this met S_cpdnew10_c0
# in these reactions = rxnnew55_c0, rxnnew58_c0, rxnnew54_c0, rxnnew59_c0, rxnnew57_c0, rxnnew60_c0
model.reactions.get_by_id('rxnnew58_c0')
showFormula(model.reactions.get_by_id('rxnnew58_c0'))
#showFormula(model.reactions.get_by_id('rxnnew55_c0'))
# rnxOfInterest = model.metabolites.get_by_id('S_cpdnew10_c0').reactions
# rnxOfInterest[0]

Reaction identifier,rxnnew58_c0
Name,Lysophosphatidylinositol synthesis
Memory address,0x7f7859c73e50
Stoichiometry,S_cpd00001_c0 + S_cpd00002_c0 + S_cpd00052_c0 + S_cpd00080_c0 + S_cpd00863_c0 + S_cpdnew10_c0 --> S_cpd00009_c0 + 2.0 S_cpd00012_c0 + S_cpd00018_c0 + S_cpd00046_c0 + S_cpd11493_c0 + S_cpdnew14_c0 H2O + ATP + CTP + Glycerol-3-phosphate + beta-D-Glucose 6-phosphate + average fatty acyl-ACP --> Phosphate + 2.0 PPi + AMP + CMP + ACP + Avg. Lysophosphatidylinositol
GPR,CDS.1625 and CDS.2164
Lower bound,0.0
Upper bound,1000.0


## look at Hydrogen unblananced reactions 

In [33]:
#rnx_h = H_unbalanced[0]
H_unbalanced[0]

Reaction identifier,rxn06118_c0
Name,Polyphosphate:D-glucose 6-phosphotransferase
Memory address,0x7f7859f96d50
Stoichiometry,S_cpd00027_c0 + S_cpd00067_c0 + S_cpdnew01_c0 <=> S_cpd00079_c0 + S_cpd00421_c0 D-Glucose + H+ + Polyphosphate <=> D-glucose-6-phosphate + Triphosphate
GPR,CDS.653
Lower bound,-1000.0
Upper bound,1000.0


In [39]:
rnx_h = H_unbalanced[0]
showFormula(rnx_h)

reactants
C6H12O6
H
HO13P4

products
C6H12O9P
HO10P3


12